# Network Analysis

StrainDesign provides canonical functions for maximizing and minimizing metabolic fluxes in network. The output format is identical to the ones of COBRApy's functions.

## Flux optimization (FBA/pFBA)

Flux balance analysis is a single linear program that optimizes a flux rate under the given steady-state network constraints. For the case of a growth-rate maximization, the problem is written as:
$$\text{maximize}\:\:\: v_{growth} \\ \text{subject to} \\ \mathbf{S\, v = 0} \\ \mathbf{lb \le v \le ub}$$
Where $ \mathbf{S}$ is the stoichiometric matrix, $\mathbf{v}$ is the vector of metabolic flux rates and $\mathbf{lb}$ and $\mathbf{ub}$ are the physiological lower and upper bounds of each flux rate. $\mathbf{S\, v = 0}$ represents all steady-state constraints, and $\mathbf{lb \le v \le ub}$ the flux ranges.

All of the following computation examples will require the COBRApy and the StrainDesign package. Here, we load both packages and the e_coli_core model from Bigg:

In [2]:
import cobra
import straindesign as sd
model = cobra.io.load_model('e_coli_core')

An FBA is launched by a single function call. By default the model's objective function is optimized. The function returns a solution object, in which the objective value and the fluxes are stored in `solution.objective_value` and `solution.fluxes`.

In [2]:
solution = sd.fba(model)

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.8739215069684305.


You may also use a custom objective (in form of a linear expression) and change the optimization sense. Here we minimize the Glucose uptake rate through the PTS:

In [3]:
solution = sd.fba(model,obj='GLCpts',obj_sense='minimize')

print(f"Minimum flux through GLCpts: {solution.objective_value}.")

Minimum flux through GLCpts: 0.4794285714285715.


We can also consider custom constraints (in this case limited oxygen uptake and an increased fixed ATP maintenance demand):

In [4]:
solution = sd.fba(model,constraints=['-EX_o2_e <= 5', 'ATPM = 20'])

print(f"Maximum growth at limited oxygen uptake and high ATP maintenance: {solution.objective_value}.")

Maximum growth at limited oxygen uptake and high ATP maintenance: 0.26305573292588313.


##### Parsimonious FBA (pFBA)

Parsimonious flux balance analysis optimizes a flux rate under the given steady-state network constraints, but also minimizes the sum of absolute fluxes to achieve this optimum. One can write:
$$\text{minimize}\:\:\:\Sigma | v_i | \\ \text{subject to} \\$$ 
$$\text{maximize}\:\:\: v_{growth} \\ \text{subject to} \\ \mathbf{S\, v = 0} \\ \mathbf{lb \le v \le ub}$$

pFBA is the simpleset way of emulating the cell's enzyme cost minimization (after an assumed growth maximization). pFBA solutions are more often "unique" than pure FBA solutions, since the outer minimization leaves fewer degrees of freedom in the solution space. StrainDesign computes pFBA solutions when you pass the 'pFBA'-argument with a value of 1. 

In [12]:
fba_sol = sd.fba(model)
pfba1_sol = sd.fba(model,pfba=1)
print(f"The sum of fluxes of the regular FBA: {sum([abs(v) for v in fba_sol.fluxes.values()])} "+\
      f"is usually higher than of the parsimoniuos FBA: {sum([abs(v) for v in pfba1_sol.fluxes.values()])}")

The sum of fluxes of the regular FBA: 2508.293334194643is usually higher than of the parsimoniuos FBA: 518.4220855176071


Likewise, it is possible to minimize the number of active reactions to attain an optimal flux distribution. We therefore use pFBA mode 2. Most of the times modes 1 and 2 yield the same results.

In [16]:
pfba1_sol = sd.fba(model,pfba=1)
pfba2_sol = sd.fba(model,pfba=2)
print(f"The number of active reactions in pFBA1: {sum([v!=0 for v in pfba1_sol.fluxes.values()])}, "+\
      f"and pFBA2: {sum([v!=0 for v in pfba2_sol.fluxes.values()])}, is often identical.")

The number of active reactions in pFBA1: 48, and pFBA2: 48, is often identical.


## Solver selection

For any type of LP or MILP optimization, 4 different sovers are supported: **GLPK** (which is built into COBRApy/optlang), **IBM CPLEX**, **Gurobi** and the open source solver **SCIP**. You can query the available solvers by accessing the set at straindesign.avail_solvers.

In [17]:
sd.avail_solvers

{'cplex', 'glpk', 'gurobi', 'scip'}

You may enforce the use of a specific solver by specifying the "solver"-keyword. To enforce the use of GLPK, use: 

In [11]:
solution = sd.fba(model, solver='cplex')

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.8739215069684826.


By default, the automatic solver selection uses COBRApy's selection. Therefore, StrainDesign will try to use the model's selected solver:

In [25]:
print(f"When the model\'s solver is \'{model.solver.configuration}', StrainDesign selects {sd.select_solver(None,model)}.")

When the model's solver is '<optlang.gurobi_interface.Configuration object at 0x000001C9B8A5A970>', StrainDesign selects gurobi.


Otherwise COBRApy's global configuration is used.

In [28]:
print(f"COBRApy\'s solver is \'{cobra.Configuration().solver.__name__}', StrainDesign selects {sd.select_solver()}.")

COBRApy's solver is 'optlang.gurobi_interface', StrainDesign selects gurobi.


## Flux variability analysis (FVA)

The fva-function determines the possible flux ranges under the given model constraints:

In [18]:
solution = sd.fva(model)
print(solution)

         minimum  maximum
PFK          0.0   176.61
PFL          0.0    40.00
PGI        -50.0    10.00
PGK        -20.0    -0.00
PGL          0.0    60.00
...          ...      ...
NADH16       0.0   120.00
NADTRHD      0.0   378.22
NH4t         0.0    10.00
O2t          0.0    60.00
PDH          0.0    40.00

[95 rows x 2 columns]


The parameters 'solver' and 'constraints' can also be used in the FVA function call. As an example, we determine the flux ranges for the case that the flux sum of PDH and PFL is smaller than 8.

In [21]:
solution = sd.fva(model, constraints='PDH + PFL <= 8')
print(solution)

         minimum     maximum
PFK          0.0  147.610000
PFL          0.0    8.000000
PGI        -50.0   10.000000
PGK        -20.0   -0.000000
PGL          0.0   60.000000
...          ...         ...
NADH16       0.0  120.000000
NADTRHD      0.0  375.220000
NH4t         0.0    8.300263
O2t          0.0   60.000000
PDH          0.0    8.000000

[95 rows x 2 columns]


## Yield optimization

Yield optmization is more than a simple FBA with a subsequent division of product and substrate flux. Consider the following example of maximizing growth and biomass yield under limited oxygen uptake.

In [68]:
numerator = 'BIOMASS_Ecoli_core_w_GAM'
denominator = '-EX_glc__D_e'
constraint = '-EX_o2_e <= 3'

sol_fba = sd.fba(model,obj='BIOMASS_Ecoli_core_w_GAM',obj_sense='maximize',constraints=constraint)
fba_yield = solution.fluxes[numerator] / -solution.fluxes['EX_glc__D_e'] 

sol_yopt = sd.yopt(model,obj_num=numerator,obj_den=denominator,obj_sense='maximize',constraints=constraint)
yopt_yield = sol_yopt.objective_value

print(f"Maximum yield (FBA): {fba_yield}.")
print(f"Maximum yield (yOpt): {yopt_yield}.")

Minimum yield (FBA): 0.031965425062067315.
Minimum yield (yOpt): 0.03629426243040193.


The best biomass yield is achieved when only respiration is used. The best growth rates use respiration and *additionally* overflow metabolism with poorer biomass yield.

The plotting chapter shows how relationships between yields and rates can be visualized.

##### Mathematical background:

Constraint-based models ($\mathbf{S\, v = 0}$, $\mathbf{lb \le v \le ub}$) can be rewritten in a single matrix-inequality term ($\mathbf{A \, x \le b}$).

With this notation, the yield optimization is a linear fractional program (LFP):
$$
\begin{array}{l c}
	\text{maximize} \; &\mathbf{\tfrac{c^\intercal x}{d^\intercal x}} \\
	\text{subject to}  &\mathbf{A \, x \le b}.
\end{array}
$$
Under the condition that the denominator term is strictly positive ($\mathbf{A\,x\le b}\, \Rightarrow\, \mathbf{d}^\intercal \mathbf{x}>\,0$), the LFP may be rewritten as an LP problem, using the Charnes-Cooper transformation. The formerly fixed boundaries $\mathbf{b}$ of the problem are then scaled by the auxiliary variable $e$ while the variable $y = \mathbf{\tfrac{c^\intercal x}{d^\intercal x}}$ expresses the original objective function:
$$
\begin{array}{l c}
	\text{maximize} \; & y  \\\\
	\text{subject to} &
	\begin{bmatrix*}[r]
		\mathbf{A~} &  \mathbf{-b} &  \mathbf{0} \\
		\mathbf{d}^\intercal &  0 		  &  0 \\
		\mathbf{c}^\intercal &  0  		  &  -1 \\
	\end{bmatrix*} 
	\begin{bmatrix}\mathbf{\tilde{x}} \\ e \\ y \end{bmatrix} 
	\begin{matrix} \le  \\ = \\ = \end{matrix} 
	\begin{bmatrix}\mathbf{0} \\ 1 \\ 0 \end{bmatrix}\\\\
	&e \ge 0.
\end{array}
$$

Solutions of $\mathbf{x}$ to the LFP (first problem) can be retrieved from a solution of the LP through $\mathbf{x}=\frac{\mathbf{\tilde{x}}}{e}$.
